In [ ]:
# Para leer datos de Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Path de la carpeta donde tenemos los datos
folder = "/content/drive/MyDrive/Colab Notebooks/BD NOVIEMBRE/"

In [ ]:
# 1. Carga de datos
df = pd.read_csv(folder + "Canciones_Spotify.csv")
df.sample(5)

In [ ]:
titulos = df.columns.tolist()
titulos

In [ ]:
selected_features = df[["danceability", "energy", "valence", "loudness"]]

In [ ]:
from sklearn.model_selection import train_test_split

# Separar características (X) y etiquetas (y)
X = selected_features
y = df["target"]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualización de la distribución de 'target'
sns.countplot(data=df, x='target')
plt.title("Distribución de 'target'")
plt.show()

In [ ]:
# Creación de Modelos de Aprendizaje Automático
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# Crear modelos
knn_model = KNeighborsClassifier()
svm_model = SVC()
dt_model = DecisionTreeClassifier()
nb_model = GaussianNB()

# Entrenar los modelos con los datos de entrenamiento
knn_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)
dt_model.fit(X_train, y_train)
nb_model.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score

# Validación cruzada k-fold
knn_scores = cross_val_score(knn_model, X_train, y_train, cv=5)
svm_scores = cross_val_score(svm_model, X_train, y_train, cv=5)
dt_scores = cross_val_score(dt_model, X_train, y_train, cv=5)
nb_scores = cross_val_score(nb_model, X_train, y_train, cv=5)

# Muestra los puntajes promedio de validación cruzada para cada modelo
print("K-Nearest Neighbors (KNN) - Puntaje promedio:", knn_scores.mean())
print("Support Vector Machines (SVM) - Puntaje promedio:", svm_scores.mean())
print("Árbol de decisión - Puntaje promedio:", dt_scores.mean())
print("Naive Bayes - Puntaje promedio:", nb_scores.mean())

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Entrenar el modelo KNN con los datos de entrenamiento
knn_model.fit(X_train, y_train) # se puede sacar ya que esta mas arriba

# Realizar predicciones en los datos de prueba
y_pred_knn = knn_model.predict(X_test)

# Matriz de confusión para el modelo KNN
cm = confusion_matrix(y_test, y_pred_knn)
sns.heatmap(cm, annot=True, fmt="d")
plt.title("Matriz de Confusión - KNN")
plt.xlabel("Predicción")
plt.ylabel("Verdadero")
plt.show()


In [ ]:
# Ajuste de Hiperparámetros para ver si podemos mejorar su rendimiento
from sklearn.model_selection import GridSearchCV

# Definir los hiperparámetros que deseas ajustar para cada modelo
knn_param_grid = {'n_neighbors': [3, 5, 7, 9]}
svm_param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
dt_param_grid = {'max_depth': [None, 5, 10, 15]}
nb_param_grid = {}  # No hay hiperparámetros para ajustar en Naive Bayes

# Crear objetos GridSearchCV para cada modelo
knn_grid = GridSearchCV(knn_model, knn_param_grid, cv=5)
svm_grid = GridSearchCV(svm_model, svm_param_grid, cv=5)
dt_grid = GridSearchCV(dt_model, dt_param_grid, cv=5)
nb_grid = nb_model  # No se ajustan hiperparámetros en Naive Bayes

# Realizar el ajuste de hiperparámetros
knn_grid.fit(X_train, y_train)
svm_grid.fit(X_train, y_train)
dt_grid.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_knn_params = knn_grid.best_params_
best_svm_params = svm_grid.best_params_
best_dt_params = dt_grid.best_params_

In [ ]:
from sklearn.ensemble import VotingClassifier

# Crear un modelo de votación mayoritaria
voting_model = VotingClassifier(estimators=[
    ('KNN', knn_model),
    ('SVM', svm_model),
    ('DecisionTree', dt_model),
    ('NaiveBayes', nb_model)
], voting='hard')  # 'hard' significa votación por mayoría

# Entrenar el modelo de votación con los datos de entrenamiento
voting_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Predicciones del modelo de votación
y_pred_voting = voting_model.predict(X_test)

# Predicciones de cada modelo individual
y_pred_knn = knn_model.predict(X_test)
y_pred_svm = svm_model.predict(X_test)
y_pred_dt = dt_model.predict(X_test)
y_pred_nb = nb_model.predict(X_test)

# Función para imprimir métricas
def print_metrics(y_true, y_pred, model_name):
    print(f"Modelo: {model_name}")
    print("Matriz de Confusión:")
    print(confusion_matrix(y_true, y_pred))
    print("Precisión:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1-Score:", f1_score(y_true, y_pred))
    print("Exactitud (Accuracy):", accuracy_score(y_true, y_pred))
    print("\n")

# Imprimir métricas para el modelo de votación
print_metrics(y_test, y_pred_voting, "Modelo de Votación")

# Imprimir métricas para cada modelo individual
print_metrics(y_test, y_pred_knn, "K-Nearest Neighbors (KNN)")
print_metrics(y_test, y_pred_svm, "Support Vector Machines (SVM)")
print_metrics(y_test, y_pred_dt, "Árbol de decisión")
print_metrics(y_test, y_pred_nb, "Naive Bayes")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predicciones del modelo de votación
y_pred_voting = voting_model.predict(X_test)

# Predicciones de cada modelo individual
y_pred_knn = knn_model.predict(X_test)
y_pred_svm = svm_model.predict(X_test)
y_pred_dt = dt_model.predict(X_test)
y_pred_nb = nb_model.predict(X_test)

def Matriz_confusion_grafica(y_true, y_pred, model_name):
    # Calcular la matriz de confusión
    cm = confusion_matrix(y_true, y_pred)

    # Visualizar la matriz de confusión con Seaborn
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"Matriz de Confusión - {model_name}")
    plt.xlabel("Predicción")
    plt.ylabel("Verdadero")
    plt.show()

Matriz_confusion_grafica(y_test, y_pred_voting, "Modelo de Votación")
Matriz_confusion_grafica(y_test, y_pred_knn, "K-Nearest Neighbors (KNN)")
Matriz_confusion_grafica(y_test, y_pred_svm, "Support Vector Machines (SVM)")
Matriz_confusion_grafica(y_test, y_pred_dt, "Árbol de decisión")
Matriz_confusion_grafica(y_test, y_pred_nb, "Naive Bayes")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calcular precisión, recall y F1-score para KNN
precision_knn = precision_score(y_test, y_pred_knn)
recall_knn = recall_score(y_test, y_pred_knn)
f1_knn = f1_score(y_test, y_pred_knn)

precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)

precision_dt = precision_score(y_test, y_pred_dt)
recall_dt = recall_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt)

precision_nb = precision_score(y_test, y_pred_nb)
recall_nb = recall_score(y_test, y_pred_nb)
f1_nb = f1_score(y_test, y_pred_nb)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Crear un DataFrame con todas las métricas para todos los modelos
all_metrics = pd.DataFrame({
    'Modelo': ['KNN', 'SVM', 'Árbol de Decisión', 'Naive Bayes'],
    'Precisión': [precision_knn, precision_svm, precision_dt, precision_nb],
    'Recall': [recall_knn, recall_svm, recall_dt, recall_nb],
    'F1-Score': [f1_knn, f1_svm, f1_dt, f1_nb]
})

# Reorganizar los datos para el lineplot
all_metrics = all_metrics.melt(id_vars='Modelo', var_name='Métrica', value_name='Valor')

# Visualización de todas las métricas en un lineplot
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.lineplot(x='Métrica', y='Valor', hue='Modelo', data=all_metrics, palette='husl', marker='o')
plt.title('Métricas para todos los modelos')
plt.ylabel('Valor')
plt.show()
